In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from time import perf_counter as time
from scipy.interpolate import RegularGridInterpolator
from matplotlib.colors import LightSource
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure
import scipy.ndimage

from pygel3d import hmesh, jupyter_display as jd
import plotly.graph_objs as go
from commons.utils import *
from medial_axis_processing import unfolding
from medial_axis_loader.shared import *
from medial_axis_loader.from_qmat import *
from medial_axis_processing.volume_sampling import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
camera = dict(
    up=dict(x=0, y=-1, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0, z=2.0)
)

def display_mesh(m, vertex_colors=None, save_html=None):
    xyz = np.array([p for p in m.positions()])
    m_tri = hmesh.Manifold(m)
    hmesh.triangulate(m_tri)
    ijk = np.array([[idx for idx in m_tri.circulate_face(f, 'v')] for f in m_tri.faces()])
    mesh = go.Mesh3d(x=xyz[:, 0], y=xyz[:, 1], z=xyz[:, 2],
                     i=ijk[:, 0], j=ijk[:, 1], k=ijk[:, 2], vertexcolor=vertex_colors, flatshading=False)

    mesh_data = [mesh]

    fig = go.Figure(data=mesh_data)
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            aspectmode="data",
            camera=camera
        ),
        width=850, height=1200
    )
    if save_html is not None:
        fig.write_html(save_html + ".html")
    else:
        fig.show()

In [4]:
# Load volumetric data
volume_data_path = 'data/vindelev/X19_resampled_uint8.nii'
niiVol = nib.load(volume_data_path)

In [5]:
affine = niiVol.affine
imgSpacing = niiVol.header['pixdim'][1:4]

# Downsample
downsample_factor = 4
affine[0, 0] = affine[0, 0] * downsample_factor
affine[1, 1] = affine[1, 1] * downsample_factor
affine[2, 2] = affine[2, 2] * downsample_factor
imgSpacing = imgSpacing * downsample_factor

vol = scipy.ndimage.zoom(niiVol.get_fdata().astype('float32'), 1 / downsample_factor, order=2)
imgDim = vol.shape

intMethod = 'linear'  # Options: "linear", "nearest", "slinear", "cubic", "quintic" and "pchip"
expVal = 0.0  # Value for extrapolation (i.e. values outside volume domain)
x = np.arange(start=0, stop=imgDim[0], step=1) * imgSpacing[0] + affine[0, 3]
y = np.arange(start=0, stop=imgDim[1], step=1) * imgSpacing[1] + affine[1, 3]
z = np.arange(start=0, stop=imgDim[2], step=1) * imgSpacing[2] + affine[2, 3]

F_vol = RegularGridInterpolator((x, y, z), vol, method=intMethod, bounds_error=False, fill_value=expVal)

In [6]:
# threshold = 130
# vertices_f, faces_f, _,_ = measure.marching_cubes(vol > threshold , 0.5)
# m = hmesh.Manifold.from_triangles(vertices_f, faces_f)
# hmesh.obj_save("x19_marching_cubes_orig.obj", m)

In [7]:
# Load medial sheet
medial_sheet = hmesh.load("data/vindelev/medial_sheet_x19_sampled_extreme.obj")
hmesh.triangulate(medial_sheet)
medial_sheet.cleanup()

# # from qmat input
# vertices, edges, faces = read_qmat("data/vindelev/x19_voxelized.ma___v_10000___e_29712___f_19715.ma")
# medial_sheet = to_medial_sheet(vertices, faces)
# medial_sheet = fix_normals(medial_sheet)
# medial_sheet = hmesh.load("data/vindelev/x19_medial_axis_qmat.obj")

In [8]:
len(medial_sheet.vertices())

497523

In [9]:
vertices = medial_sheet.positions()
normals = np.array([medial_sheet.vertex_normal(vid) for vid in medial_sheet.vertices()])

vertices_ft = vertices @ affine[0:3, 0:3] + np.transpose(affine[0:3, 3])
normals_ft = normals @ affine[0:3, 0:3]

In [10]:
sampling_distance = min(imgSpacing)
num_samples = 20

num_vertices = vertices_ft.shape[0]
sample_indices = np.linspace(-num_samples, num_samples - 1, num_samples)

j_factors = sample_indices[:, None] * sampling_distance

samples_ft = vertices_ft[None, :, :] + normals_ft[None, :, :] * j_factors[:, None, None]

samples_ft_reshaped = samples_ft.reshape(-1, samples_ft.shape[-1])

intensities = F_vol(samples_ft_reshaped)

intensities_reshaped = intensities.reshape(num_samples, num_vertices)
max_intensities = np.max(intensities_reshaped, axis=0)

In [11]:
# Convert to RGB color
rgbColor = np.transpose(np.tile(max_intensities.astype('uint8'),(3,1))) / 255

In [12]:
display_mesh(medial_sheet, vertex_colors=rgbColor, save_html="results/vindelev/medial_sheet")

In [13]:
unfolded = hmesh.Manifold(medial_sheet)
unfolded_pos = unfolding.get_unfolded_sheet_positions(None, medial_sheet)
unfolded.positions()[:] = unfolded_pos

In [14]:
display_mesh(unfolded, vertex_colors=rgbColor, save_html="results/vindelev/unfolded")

In [15]:
np.savetxt("results/vindelev/intensities_medial_sheet_x19_sampled_extreme.txt", rgbColor)